In [ ]:
import os
os.chdir('../..')

In [ ]:
import spacy
import srsly

In [ ]:
import pandas as pd

In [ ]:
concepts = pd.read_csv("assets/concepts-source-index.tsv", sep="\t")
countries = pd.read_csv("assets/countries-source-index.tsv", sep="\t")

In [ ]:
concepts

In [ ]:
countries

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
import re
srsly.write_jsonl('data/linkedsdg_concepts.jsonl', ({"label":"CONCEPT","pattern":[{"LOWER": tok.lower_} for tok in concept]} for concept in nlp.tokenizer.pipe(concepts.label.str.lower())))
concepts.label.str.lower().to_csv("../assets/linkedsdg_concepts.txt", index=False, header=False)

In [ ]:
spacy.displacy.serve(nlp("I live in south of Europe."), style='ent')

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

In [ ]:
patterns = list(nlp.tokenizer.pipe(concepts.label.str.lower()))

In [ ]:
matcher.add("Concepts", patterns)

In [ ]:
doc = nlp("I'm not sure about the 02 Crimea questions...")

In [ ]:
doc[6].lemma_

In [ ]:
for match_id, start, end in matcher(doc):
    print("Matched based on 02 lowercase token text:", doc[start:end])

In [ ]:
spacy.displacy.serve(doc)

In [ ]:
def clean_nounchunk(nc):
    return ' '.join([tok.text.lower().strip() for tok in nc if tok.pos_ not in ['DET', 'NUM', 'PUNCT']]).strip()

In [ ]:
texts = []
for item in srsly.read_jsonl('../assets/annotations_raw.jsonl'):
    text = item['text']
    doc = nlp(text)
    texts.extend([[clean_nounchunk(nc) for nc in sent.noun_chunks] for sent in doc.sents])


In [ ]:
for item in srsly.read_jsonl('../assets/documents.jsonl'):
    text = item['text']
    doc = nlp(text)
    texts.extend([[clean_nounchunk(nc) for nc in sent.noun_chunks] for sent in doc.sents])


In [ ]:
len(texts)*0.001

In [ ]:
stop_words = nlp.Defaults.stop_words
stop_words |= set(countries.label.str.lower())
stop_words |= set(concepts.label.str.lower())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=3, max_df=0.005, stop_words="english", ngram_range=(1,4), analyzer=lambda x:[w.lower() for w in x if w.lower() not in stop_words and not re.match(r'.*\d+', w)])
X = vectorizer.fit_transform(texts)
vectorizer.get_feature_names()

In [ ]:
sdgio = pd.read_json('../../sdgio/sdgio.json', encoding="cp1252", )

In [ ]:
sdgio

In [ ]:
import json
with open('../../sdgio/sdgio.json', encoding="cp1252") as f:
    sdgio = json.load(f, )
    
nodes = sdgio['graphs'][0]['nodes']

In [ ]:
df = pd.DataFrame(nodes)
df.iloc[:10]

In [ ]:
df[df.type == 'INDIVIDUAL'][-10:]

In [ ]:
df.id.str.split('.org').apply(lambda x: x[0]).unique()